In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import pandas as pd
import os
from training.datagenerator import DataGenerator
from models import pointnet_seg
import numpy as np
import json
from sklearn.model_selection import train_test_split
from vedo import show, Points, settings

settings.default_backend = 'k3d'

2023-08-05 14:44:32.371706: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 14:44:32.408521: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-05 14:44:32.409379: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-05 14:44:32.995449: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
with open("data/Teeth3DS_train_test_split/training_lower.txt", "r") as f:
    train_ex = f.read().splitlines()

In [7]:
with open("data/Teeth3DS_train_test_split/testing_lower.txt", "r") as f:
    test_ex = f.read().splitlines()

In [8]:
LOWER_PATH = "data/lower"
folders = os.listdir(LOWER_PATH)
obj_files = [
    os.path.join(LOWER_PATH, f, f"{f}_lower.obj")
    for f in folders
    if f"{f}_lower" in train_ex
]
label_files = [
    os.path.join(LOWER_PATH, f, f"{f}_lower.json")
    for f in folders
    if f"{f}_lower" in train_ex
]
assert len(obj_files) == len(label_files)

In [9]:
df = pd.DataFrame({"obj_file": obj_files, "label_file": label_files})


In [10]:
df = pd.DataFrame({"obj_file": obj_files, "label_file": label_files})
labels_list = []
for row in df.itertuples():
    with open(row.label_file, "r") as file:
        labels = json.load(file)
    labels_list.append(labels["labels"])
df["labels"] = labels_list

In [11]:
with open("labels.json", "r") as f:
    teeth_name_label_dict = json.load(f)

LABELS_TO_IDENTIFY = [teeth_name_label_dict[name] for name in ["LeftMandibularSecondMolar"]]

In [12]:
df['has_selected_teeth'] = df['labels'].apply(lambda labels: set(LABELS_TO_IDENTIFY).issubset(labels))


In [13]:
np.unique(df.has_selected_teeth.values, return_counts=True)

(array([False,  True]), array([ 77, 223]))

In [14]:
from sklearn.model_selection import train_test_split
# Split data to train
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
)

In [50]:
df_to_train = df[df["has_selected_teeth"]]


In [52]:
df_to_train["selected_teeth"]=df_to_train["labels"].apply(lambda labels:[label if label in LABELS_TO_IDENTIFY else 0 for label in labels])

/tmp/ipykernel_7379/2386187063.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_train["selected_teeth"]=df_to_train["labels"].apply(lambda labels:[label if label in LABELS_TO_IDENTIFY else 0 for label in labels])


In [54]:
df_to_train["labels"]

2      [32, 0, 41, 36, 36, 31, 33, 47, 0, 0, 42, 45, ...
3      [34, 0, 0, 43, 0, 0, 32, 0, 0, 0, 35, 47, 45, ...
5      [0, 46, 47, 0, 0, 0, 0, 45, 42, 44, 45, 34, 37...
8      [0, 0, 0, 0, 0, 0, 36, 34, 0, 45, 35, 0, 32, 0...
11     [48, 0, 43, 0, 41, 0, 0, 0, 0, 44, 0, 0, 0, 0,...
                             ...                        
295    [36, 0, 0, 0, 46, 45, 0, 46, 32, 0, 36, 0, 42,...
296    [35, 0, 0, 37, 45, 36, 41, 0, 46, 0, 45, 31, 3...
297    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34, 0, 31...
298    [45, 0, 47, 45, 31, 43, 45, 33, 45, 45, 46, 0,...
299    [0, 0, 0, 47, 0, 0, 0, 34, 46, 0, 35, 0, 46, 0...
Name: labels, Length: 223, dtype: object

In [16]:
df.labels[0]

[45,
 41,
 33,
 45,
 0,
 0,
 0,
 0,
 45,
 33,
 45,
 31,
 0,
 45,
 0,
 42,
 36,
 36,
 32,
 0,
 45,
 44,
 0,
 0,
 32,
 36,
 32,
 42,
 0,
 41,
 0,
 35,
 46,
 43,
 35,
 0,
 32,
 45,
 35,
 35,
 34,
 34,
 36,
 0,
 42,
 0,
 42,
 36,
 43,
 35,
 36,
 36,
 0,
 0,
 0,
 0,
 45,
 32,
 45,
 0,
 46,
 45,
 36,
 0,
 36,
 43,
 33,
 0,
 44,
 0,
 0,
 0,
 41,
 42,
 34,
 35,
 32,
 35,
 36,
 0,
 0,
 42,
 46,
 44,
 0,
 0,
 42,
 34,
 0,
 34,
 32,
 0,
 35,
 32,
 0,
 32,
 0,
 36,
 42,
 0,
 45,
 36,
 0,
 41,
 32,
 44,
 0,
 43,
 36,
 0,
 41,
 0,
 41,
 0,
 31,
 0,
 0,
 0,
 35,
 35,
 43,
 35,
 46,
 35,
 36,
 0,
 44,
 34,
 41,
 45,
 36,
 33,
 32,
 0,
 41,
 45,
 0,
 44,
 31,
 36,
 36,
 43,
 0,
 43,
 46,
 35,
 31,
 33,
 36,
 0,
 31,
 36,
 35,
 0,
 43,
 31,
 36,
 0,
 31,
 36,
 0,
 45,
 32,
 43,
 32,
 0,
 45,
 45,
 41,
 0,
 46,
 0,
 32,
 0,
 35,
 0,
 34,
 36,
 41,
 46,
 36,
 36,
 34,
 0,
 46,
 35,
 36,
 36,
 0,
 41,
 0,
 0,
 0,
 31,
 0,
 0,
 33,
 35,
 31,
 45,
 0,
 0,
 43,
 0,
 32,
 34,
 46,
 0,
 0,
 0,
 0,
 31,
 31,
 35

In [6]:
# how many unique labels are there?

unique_labels = []

for f in label_files:
    with open(f, "r") as file:
        labels = json.load(file)
    # Store class
    unique_labels.append(np.unique(labels["labels"]))

unique_labels = np.unique(np.concatenate(unique_labels))
print(f"Num unique {len(unique_labels)}, Unique {unique_labels}")

Num unique 17, Unique [ 0 31 32 33 34 35 36 37 38 41 42 43 44 45 46 47 48]


In [7]:
x_train, x_val, y_train, y_val = train_test_split(
    obj_files, label_files, test_size=0.2, random_state=42
)

In [8]:
training_generator = DataGenerator(x_train, y_train, unique_labels=unique_labels)
validation_generator = DataGenerator(x_val, y_val, unique_labels=unique_labels)

In [9]:
i = 0

for t in training_generator:
    print(t[0].shape)
    print(t[1].shape)
    i += 1
    print(np.unique(t[1]))
    if i == 5:
        break

(1, 10000, 3)
(1, 10000, 17)
[0. 1.]
(1, 10000, 3)
(1, 10000, 17)
[0. 1.]
(1, 10000, 3)
(1, 10000, 17)
[0. 1.]
(1, 10000, 3)
(1, 10000, 17)
[0. 1.]
(1, 10000, 3)
(1, 10000, 17)
[0. 1.]


In [10]:
model = pointnet_seg.get_shape_segmentation_model(10000, num_classes=len(unique_labels))

2023-08-02 22:41:46.087776: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-02 22:41:46.088631: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 3)]            0         []                            
                                                                                                  
 input_transformation_block  (None, None, 64)             256       ['input_1[0][0]']             
 _1_conv (Conv1D)                                                                                 
                                                                                                  
 input_transformation_block  (None, None, 64)             256       ['input_transformation_block_1
 _1_batch_norm (BatchNormal                                         _conv[0][0]']                 
 ization)                                                                                     

                                                                                                  
 input_transformation_block  (None, None, 128)            8320      ['input_transformation_block_1
 _2_conv (Conv1D)                                                   _relu[0][0]']                 
                                                                                                  
 input_transformation_block  (None, None, 128)            512       ['input_transformation_block_2
 _2_batch_norm (BatchNormal                                         _conv[0][0]']                 
 ization)                                                                                         
                                                                                                  
 input_transformation_block  (None, None, 128)            0         ['input_transformation_block_2
 _2_relu (Activation)                                               _batch_norm[0][0]']           
          

In [12]:
model.fit(
    training_generator, validation_data=validation_generator, verbose=1, epochs=1
)

240/240 [==============================] - 263s 1s/step - loss: 0.8312 - accuracy: 0.6822 - val_loss: 2.6910 - val_accuracy: 0.5040


In [13]:
test_obj_files = [
    os.path.join(LOWER_PATH, f, f"{f}_lower.obj")
    for f in folders
    if f"{f}_lower" in test_ex
]
test_label_files = [
    os.path.join(LOWER_PATH, f, f"{f}_lower.json")
    for f in folders
    if f"{f}_lower" in test_ex
]
assert len(test_obj_files) == len(test_label_files)

In [14]:
test_generator = DataGenerator(test_obj_files, test_label_files, unique_labels=unique_labels)

In [15]:
for t in test_generator:
    inp = t[0]
    result = model.predict(t[0])
    prediction = np.argmax(result, axis=-1)
    break

1/1 [==============================] - 1s 556ms/step


In [ ]:
inp.shape

(1, 10000, 3)

In [ ]:
prediction.shape

(1, 10000)

In [ ]:

# Create a Points object for visualization
pointcloud = Points(inp[0])
show(pointcloud)